In [1]:
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

In [2]:
from ronek.argon_plasma.species import Species
from ronek.argon_plasma.mixture import Mixture
from ronek.argon_plasma.kinetics import Kinetics
import tensorflow as tf
import dill as pickle
import numpy as np

In [3]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [4]:
path = path_to_dtb + "/species/"
species = "Arp"

In [5]:
species_order = ("Ar", "Arp", "em")
species = {k: path + "/" + k + ".json" for k in species_order}

In [6]:
mix = Mixture(species, species_order)
mix.build()

In [7]:
p = 1000.0
T = 10000.0
rho, w = mix.get_init_composition(p, T, noise=False)

In [ ]:
rho, w, np.sum(w), np.sum(mix.species["Arp"].x)-mix.species["em"].x

In [ ]:
Ti = np.random.rand(1)*10000.0
Tei = np.random.rand(1)*3000.0
Ti, Tei

In [ ]:
mix.update_composition(w, rho)
mix.update_species_thermo(Ti, Tei)
mix.update_mixture_thermo()
mix.cv_h, mix.e, mix.e_e, mix.e_int_h, mix.hf_h

In [ ]:
T, Te = mix.compute_temp(mix.e, mix.e_e)
T, Te

In [12]:
kinpath = path_to_dtb + "/rates/kin_fit.p"
# pickle.load(open(kinpath, "rb"))

In [13]:
kin = Kinetics(mix, kinpath, use_fit=False)

In [14]:
kin.update(T, Te)

In [ ]:
kin.rates["EN"], kin.rates["EI"]

In [ ]:
tf.nest.map_structure(np.shape, kin.rates)

In [17]:
kin2 = Kinetics(mix, kinpath, use_fit=True)

In [18]:
kin2.update(T, Te)

In [ ]:
kin2.rates["EN"], kin2.rates["EI"]

In [20]:
# for Ti in np.linspace(500, 50000, 100):
#     kin.update(T, Ti)
#     k1 = kin.rates["EN"]
#     kin2.update(T, Ti)
#     k2 = kin2.rates["EN"]
#     print(np.abs(k1-k2)/k1*100)

In [21]:
from ronek.argon_plasma.model import Model

In [22]:
model = Model(
  species={k: path + "/" + k + ".json" for k in species_order},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p"
)

In [ ]:
x = np.concatenate([w, mix.e, mix.e_e])
x, rho

In [ ]:
model.mix.de["in"]

In [ ]:
model._fun(0.0, x, rho)

In [26]:
kops = model._compose_kin_ops(T, Te)

In [27]:
nn = model.mix.species["Ar"].n
ni = model.mix.species["Arp"].n
ne = model.mix.species["em"].n

In [28]:
kf = model.kin.rates["EXe"]["fwd"]
kb = model.kin.rates["EXe"]["bwd"]

In [ ]:
f1 = model.source_term_exe(nn, ne, kf, kb)
f1

In [ ]:
f2 = model.source_term_exe_vec(nn, ne, kf, kb)
f2

In [ ]:
100*np.abs(f1-f2)/np.abs(f1)

In [255]:
kf = model.kin.rates["Ie"]["fwd"]
kb = model.kin.rates["Ie"]["bwd"]

In [ ]:
f1 = model.source_term_ie(nn, ni, ne, kf, kb)
f1

In [ ]:
f2 = model.source_term_ie_vec(nn, ni, ne, kf, kb)
f2

In [ ]:
100*np.abs(f1-f2)/np.abs(f1)